In [1]:
import pandas as pd

# Load the dataset
file_path = "C:\Capstone_project_(8th sem)\MindSight\Dataset\DASS.csv"
df = pd.read_csv(file_path)

# Display basic info and first few rows
df_info = df.info()
df_head = df.head()

df_info, df_head

<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Susheeth G\AppData\Local\Temp\ipykernel_19204\4180196248.py:4: SyntaxWarning: invalid escape sequence '\C'
  file_path = "C:\Capstone_project_(8th sem)\MindSight\Dataset\DASS.csv"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Q1_1              1812 non-null   int64
 1   Q1_2              1812 non-null   int64
 2   Q1_3              1812 non-null   int64
 3   Q1_4              1812 non-null   int64
 4   Q1_5              1812 non-null   int64
 5   Q1_6              1812 non-null   int64
 6   Q3_1_S1           1812 non-null   int64
 7   Q3_2_S2           1812 non-null   int64
 8   Q3_3_S3           1812 non-null   int64
 9   Q3_4_S4           1812 non-null   int64
 10  Q3_5_S5           1812 non-null   int64
 11  Q3_6_S6           1812 non-null   int64
 12  Q3_7_S7           1812 non-null   int64
 13  Stress_Score      1812 non-null   int64
 14  Stress_Level      1812 non-null   int64
 15  Q3_8_A1           1812 non-null   int64
 16  Q3_9_A2           1812 non-null   int64
 17  Q3_10_A3          1812 non-null  

(None,
    Q1_1  Q1_2  Q1_3  Q1_4  Q1_5  Q1_6  Q3_1_S1  Q3_2_S2  Q3_3_S3  Q3_4_S4  \
 0    21     2     0     4     4     1        3        3        3        3   
 1    18     2     0     4     4     1        3        3        3        3   
 2    40     2     1     2     1     1        3        3        1        2   
 3    24     1     0     4     2     0        3        3        1        2   
 4    50     1     1     2     2     0        3        3        3        2   
 
    ...  Anxiety_Level  Q3_15_D1  Q3_16_D2  Q3_17_D3  Q3_18_D4  Q3_19_D5  \
 0  ...              3         0         3         3         2         3   
 1  ...              2         0         2         3         0         0   
 2  ...              5         2         1         1         2         2   
 3  ...              5         2         1         1         2         2   
 4  ...              2         1         0         0         1         2   
 
    Q3_20_D6  Q3_21_D7  Depression_Score  Depression_Level  
 0  

In [2]:
# Step 1: Remove duplicates
df_cleaned = df.drop_duplicates()

In [3]:
# Step 2: Validate that all mental health question responses are in range 0–3
question_cols = [col for col in df.columns if col.startswith("Q3_")]
valid_range_check = df_cleaned[question_cols].applymap(lambda x: 0 <= x <= 3).all()

C:\Users\Susheeth G\AppData\Local\Temp\ipykernel_19204\3704412488.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  valid_range_check = df_cleaned[question_cols].applymap(lambda x: 0 <= x <= 3).all()


In [4]:
# Step 3: Check if any invalid entries exist
invalid_entries = ~valid_range_check

In [5]:
# Step 4: Report the number of duplicates removed and any invalid entries
duplicates_removed = df.shape[0] - df_cleaned.shape[0]
invalid_questions = valid_range_check[~valid_range_check].index.tolist()

duplicates_removed, invalid_questions

(65, [])

In [6]:
from sklearn.preprocessing import StandardScaler

# Step 1: Define feature columns
# Use only the Q3_* question columns (21 features) for prediction
stress_cols = [f"Q3_{i}_S{i}" for i in range(1, 8)]
anxiety_cols = [f"Q3_{i}_A{i-7}" for i in range(8, 15)]
depression_cols = [f"Q3_{i}_D{i-14}" for i in range(15, 22)]

In [7]:
# Step 2: Extract features and labels for each category
X_stress = df_cleaned[stress_cols]
y_stress = df_cleaned["Stress_Level"]

X_anxiety = df_cleaned[anxiety_cols]
y_anxiety = df_cleaned["Anxiety_Level"]

X_depression = df_cleaned[depression_cols]
y_depression = df_cleaned["Depression_Level"]

In [8]:
# Step 3: Scale features
scaler = StandardScaler()
X_stress_scaled = scaler.fit_transform(X_stress)
X_anxiety_scaled = scaler.fit_transform(X_anxiety)
X_depression_scaled = scaler.fit_transform(X_depression)


In [9]:
# Confirm shapes and preview scaled data
X_stress_scaled.shape, X_anxiety_scaled.shape, X_depression_scaled.shape

((1747, 7), (1747, 7), (1747, 7))

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Function to train and evaluate SVM model
def train_svm(X, y, label):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train the model
    model = SVC(kernel='rbf', C=1, gamma='scale')
    model.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    matrix = confusion_matrix(y_test, y_pred)
    
    return model, report, matrix

# Train and evaluate SVM for each mental health category
stress_model, stress_report, stress_matrix = train_svm(X_stress_scaled, y_stress, "Stress")
anxiety_model, anxiety_report, anxiety_matrix = train_svm(X_anxiety_scaled, y_anxiety, "Anxiety")
depression_model, depression_report, depression_matrix = train_svm(X_depression_scaled, y_depression, "Depression")

stress_report, anxiety_report, depression_report


({'1': {'precision': 0.9932885906040269,
   'recall': 1.0,
   'f1-score': 0.9966329966329966,
   'support': 148.0},
  '2': {'precision': 0.9736842105263158,
   'recall': 0.9736842105263158,
   'f1-score': 0.9736842105263158,
   'support': 76.0},
  '3': {'precision': 0.9764705882352941,
   'recall': 0.9540229885057471,
   'f1-score': 0.9651162790697675,
   'support': 87.0},
  '4': {'precision': 0.9459459459459459,
   'recall': 0.9722222222222222,
   'f1-score': 0.958904109589041,
   'support': 36.0},
  '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3.0},
  'accuracy': 0.98,
  'macro avg': {'precision': 0.9778778670623165,
   'recall': 0.979985884250857,
   'f1-score': 0.978867519163624,
   'support': 350.0},
  'weighted avg': {'precision': 0.9800391618283446,
   'recall': 0.98,
   'f1-score': 0.9799638506455963,
   'support': 350.0}},
 {'1': {'precision': 1.0,
   'recall': 0.984375,
   'f1-score': 0.9921259842519685,
   'support': 128.0},
  '2': {'precision': 0.97297

In [12]:
import joblib

# Save each model to disk
joblib.dump(stress_model, "svm_stress_model.pkl")
joblib.dump(anxiety_model, "svm_anxiety_model.pkl")
joblib.dump(depression_model, "svm_depression_model.pkl")


['svm_depression_model.pkl']

In [13]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.1, 0.01],
    'kernel': ['rbf', 'poly', 'linear']
}

# Function to run GridSearchCV
def optimize_svm(X, y):
    svc = SVC()
    grid = GridSearchCV(svc, param_grid, cv=5, verbose=2, n_jobs=-1)
    grid.fit(X, y)
    return grid.best_estimator_, grid.best_params_, grid.best_score_

# Run optimization for Stress (as an example)
best_stress_model, best_stress_params, best_stress_score = optimize_svm(X_stress_scaled, y_stress)

best_stress_params, best_stress_score


Fitting 5 folds for each of 27 candidates, totalling 135 fits


({'C': 1, 'gamma': 'scale', 'kernel': 'linear'}, np.float64(1.0))

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd


# Define question sets
stress_cols = [f"Q3_{i}_S{i}" for i in range(1, 8)]
anxiety_cols = [f"Q3_{i}_A{i-7}" for i in range(8, 15)]
depression_cols = [f"Q3_{i}_D{i-14}" for i in range(15, 22)]

# Extract features and labels
X_stress, y_stress = df[stress_cols], df["Stress_Level"]
X_anxiety, y_anxiety = df[anxiety_cols], df["Anxiety_Level"]
X_depression, y_depression = df[depression_cols], df["Depression_Level"]

# Scale features
scaler = StandardScaler()
X_stress = scaler.fit_transform(X_stress)
X_anxiety = scaler.fit_transform(X_anxiety)
X_depression = scaler.fit_transform(X_depression)

# Define parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.1, 0.01],
    'kernel': ['rbf', 'poly', 'linear']
}

def optimize_svm(X, y):
    model = SVC()
    grid = GridSearchCV(model, param_grid, cv=5, verbose=2, n_jobs=-1)
    grid.fit(X, y)
    return grid.best_estimator_, grid.best_params_, grid.best_score_

# Run optimization for each model
best_stress_model, stress_params, stress_score = optimize_svm(X_stress, y_stress)
best_anxiety_model, anxiety_params, anxiety_score = optimize_svm(X_anxiety, y_anxiety)
best_depression_model, depression_params, depression_score = optimize_svm(X_depression, y_depression)

# Save models
import joblib
joblib.dump(best_stress_model, "svm_stress_model_optimized.pkl")
joblib.dump(best_anxiety_model, "svm_anxiety_model_optimized.pkl")
joblib.dump(best_depression_model, "svm_depression_model_optimized.pkl")


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits


['svm_depression_model_optimized.pkl']

In [16]:
import joblib

# Load the saved model
model = joblib.load("svm_stress_model_optimized.pkl")  # or any other .pkl file


In [18]:
from sklearn.preprocessing import StandardScaler
import pandas as pd



# Select the same question columns used for training
stress_cols = [f"Q3_{i}_S{i}" for i in range(1, 8)]
X_stress = df[stress_cols]

# Standardize the features (important!)
scaler = StandardScaler()
X_stress_scaled = scaler.fit_transform(X_stress)


In [19]:
# True labels
y_stress = df["Stress_Level"]

# Predict
y_pred = model.predict(X_stress_scaled)

# Evaluate
from sklearn.metrics import classification_report
print(classification_report(y_stress, y_pred))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00       794
           2       1.00      1.00      1.00       422
           3       1.00      1.00      1.00       354
           4       1.00      1.00      1.00       183
           5       1.00      1.00      1.00        59

    accuracy                           1.00      1812
   macro avg       1.00      1.00      1.00      1812
weighted avg       1.00      1.00      1.00      1812



In [ ]:

sample = [[3, 3, 0, 0, 0, 0, 3]]
sample_scaled = scaler.transform(sample)
prediction = model.predict(sample_scaled)

print("Predicted Stress Level:", prediction[0])


Predicted Stress Level: 2


c:\Capstone_project_(8th sem)\MindSight\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [22]:


# Load the optimized anxiety model
model = joblib.load("svm_anxiety_model_optimized.pkl")


# Extract Anxiety question columns (Q3_8 to Q3_14)
anxiety_cols = [f"Q3_{i}_A{i-7}" for i in range(8, 15)]
X_anxiety = df[anxiety_cols]
y_anxiety = df["Anxiety_Level"]

# Standardize features (same as during training)
scaler = StandardScaler()
X_anxiety_scaled = scaler.fit_transform(X_anxiety)

# Predict using the loaded model
y_pred = model.predict(X_anxiety_scaled)

# Evaluate the predictions
print("🔍 Anxiety Model Classification Report:\n")
print(classification_report(y_anxiety, y_pred))


🔍 Anxiety Model Classification Report:

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       739
           2       1.00      1.00      1.00       364
           3       1.00      1.00      1.00       328
           4       1.00      1.00      1.00       261
           5       1.00      1.00      1.00       120

    accuracy                           1.00      1812
   macro avg       1.00      1.00      1.00      1812
weighted avg       1.00      1.00      1.00      1812



In [29]:
# Example: A sample user who answered "2" (OFTEN) to all anxiety questions
sample_input = [[1, 2, 0, 1, 0, 0, 0]]  # 7 anxiety questions
sample_scaled = scaler.transform(sample_input)
predicted_level = model.predict(sample_scaled)

print("\nPredicted Anxiety Level for sample input:", predicted_level[0])



Predicted Anxiety Level for sample input: 2


c:\Capstone_project_(8th sem)\MindSight\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [30]:

# Load the optimized depression model
model = joblib.load("svm_depression_model_optimized.pkl")



# Extract Depression question columns (Q3_15 to Q3_21)
depression_cols = [f"Q3_{i}_D{i-14}" for i in range(15, 22)]
X_depression = df[depression_cols]
y_depression = df["Depression_Level"]

# Standardize features (same as during training)
scaler = StandardScaler()
X_depression_scaled = scaler.fit_transform(X_depression)

# Predict using the loaded model
y_pred = model.predict(X_depression_scaled)

# Evaluate the predictions
print("🧠 Depression Model Classification Report:\n")
print(classification_report(y_depression, y_pred))


🧠 Depression Model Classification Report:

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       684
           2       1.00      1.00      1.00       371
           3       1.00      1.00      1.00       319
           4       1.00      1.00      1.00       273
           5       1.00      1.00      1.00       165

    accuracy                           1.00      1812
   macro avg       1.00      1.00      1.00      1812
weighted avg       1.00      1.00      1.00      1812



In [41]:
# Example: A sample user who answered "1" (SOMETIMES) to all depression questions
sample_input = [[1, 3, 1, 1, 1, 1, 1]]  # 7 depression questions
sample_scaled = scaler.transform(sample_input)
predicted_level = model.predict(sample_scaled)

print("\nPredicted Depression Level for sample input:", predicted_level[0])



Predicted Depression Level for sample input: 3


c:\Capstone_project_(8th sem)\MindSight\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


different data set

In [ ]:
model = joblib.load("svm_depression_model.pkl")



# Extract Depression question columns (Q3_15 to Q3_21)
depression_cols = [f"Q3_{i}_D{i-14}" for i in range(15, 22)]
X_depression = df[depression_cols]
y_depression = df["Depression_Level"]

# Standardize features (same as during training)
scaler = StandardScaler()
X_depression_scaled = scaler.fit_transform(X_depression)

# Predict using the loaded model
y_pred = model.predict(X_depression_scaled)

# Evaluate the predictions
print("🧠 Depression Model Classification Report:\n")
print(classification_report(y_depression, y_pred))

🧠 Depression Model Classification Report:

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       684
           2       1.00      0.99      1.00       371
           3       0.99      0.98      0.98       319
           4       0.97      0.97      0.97       273
           5       0.95      0.99      0.97       165

    accuracy                           0.99      1812
   macro avg       0.98      0.99      0.98      1812
weighted avg       0.99      0.99      0.99      1812



In [47]:
sample_input = [[0, 3, 2, 1, 0, 0, 1]]  # 7 depression questions
sample_scaled = scaler.transform(sample_input)
predicted_level = model.predict(sample_scaled)

print("\nPredicted Depression Level for sample input:", predicted_level[0])



Predicted Depression Level for sample input: 2


c:\Capstone_project_(8th sem)\MindSight\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
